In [60]:
# STEP 1: Import the necessary modules.
import os
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import RunningMode

In [102]:
# STEP 2: Setup HandLandmarker configuration
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2, running_mode=RunningMode.VIDEO)

I0000 00:00:1749818089.506500 55382079 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1749818089.512549 55534899 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749818089.518071 55534899 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
# STEP 4: Generate landmarks for each frame of every video
for root, _, files in os.walk("../food/data"):
    for filename in files:  # loop through files in the current directory
        capture = cv2.VideoCapture(os.path.join(root, filename))
        detector = vision.HandLandmarker.create_from_options(options)
        detection_results = []
        frameNr = 0
        while True:
            ret, frame = capture.read()
            if not ret:
                break
            # Convert the frame to a MediaPipe Image.
            image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            # Detect hand landmarks in the current frame.
            detection_result = detector.detect_for_video(image, frameNr)
            csv = []

            # Collect landmarks and handedness for each detected hand.
            for idx in range(len(detection_result.hand_landmarks)):
                handedness = detection_result.handedness[idx][0].category_name
                landmarks = detection_result.hand_landmarks[idx]
                csv.append("\n".join(map(lambda x: f"{handedness}, x:{x.x}, y:{x.y}, z:{x.z}", landmarks)))

            # Write the landmarks to a CSV file.
            try:
                os.mkdir(os.path.join(root, "landmarks"))
            except FileExistsError:
                pass
            with open(os.path.join(root, f"landmarks/{filename}_landmarks.csv"), "a") as f:
                f.write(f"frame {frameNr} \n")
                f.write("\n".join(csv))
                f.write("\n")

            frameNr = frameNr + 1
            print(f"Processed {filename} frame {frameNr}")
        capture.release()

OpenCV: Couldn't read video stream from file "../food/data/.DS_Store"
I0000 00:00:1749821390.480751 55382079 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1749821390.487599 55646925 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749821390.492400 55646929 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1749821390.503282 55382079 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1749821390.509363 55646952 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749821390.514205 55646952 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disa

Processed apfel_test_1.mp4 frame 1
Processed apfel_test_1.mp4 frame 2
Processed apfel_test_1.mp4 frame 3
Processed apfel_test_1.mp4 frame 4
Processed apfel_test_1.mp4 frame 5
Processed apfel_test_1.mp4 frame 6
Processed apfel_test_1.mp4 frame 7
Processed apfel_test_1.mp4 frame 8
Processed apfel_test_1.mp4 frame 9
Processed apfel_test_1.mp4 frame 10
Processed apfel_test_1.mp4 frame 11
Processed apfel_test_1.mp4 frame 12
Processed apfel_test_1.mp4 frame 13
Processed apfel_test_1.mp4 frame 14
Processed apfel_test_1.mp4 frame 15
Processed apfel_test_1.mp4 frame 16
Processed apfel_test_1.mp4 frame 17
Processed apfel_test_1.mp4 frame 18
Processed apfel_test_1.mp4 frame 19
Processed apfel_test_1.mp4 frame 20
Processed apfel_test_1.mp4 frame 21
Processed apfel_test_1.mp4 frame 22
Processed apfel_test_1.mp4 frame 23
Processed apfel_test_1.mp4 frame 24
Processed apfel_test_1.mp4 frame 25
Processed apfel_test_1.mp4 frame 26
Processed apfel_test_1.mp4 frame 27
Processed apfel_test_1.mp4 frame 28
P

I0000 00:00:1749821392.519693 55382079 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1749821392.525127 55647060 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749821392.529513 55647068 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processed apfel_test_0.mp4 frame 1
Processed apfel_test_0.mp4 frame 2
Processed apfel_test_0.mp4 frame 3
Processed apfel_test_0.mp4 frame 4
Processed apfel_test_0.mp4 frame 5
Processed apfel_test_0.mp4 frame 6
Processed apfel_test_0.mp4 frame 7
Processed apfel_test_0.mp4 frame 8
Processed apfel_test_0.mp4 frame 9
Processed apfel_test_0.mp4 frame 10
Processed apfel_test_0.mp4 frame 11
Processed apfel_test_0.mp4 frame 12
Processed apfel_test_0.mp4 frame 13
Processed apfel_test_0.mp4 frame 14
Processed apfel_test_0.mp4 frame 15
Processed apfel_test_0.mp4 frame 16
Processed apfel_test_0.mp4 frame 17
Processed apfel_test_0.mp4 frame 18
Processed apfel_test_0.mp4 frame 19
Processed apfel_test_0.mp4 frame 20
Processed apfel_test_0.mp4 frame 21
Processed apfel_test_0.mp4 frame 22
Processed apfel_test_0.mp4 frame 23
Processed apfel_test_0.mp4 frame 24
Processed apfel_test_0.mp4 frame 25
Processed apfel_test_0.mp4 frame 26
Processed apfel_test_0.mp4 frame 27
Processed apfel_test_0.mp4 frame 28
P

OpenCV: Couldn't read video stream from file "../food/data/test/apfel/landmarks/apfel_test_1.mp4_landmarks.csv"
I0000 00:00:1749821393.242454 55382079 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1749821393.247780 55647092 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749821393.251624 55647092 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
OpenCV: Couldn't read video stream from file "../food/data/test/apfel/landmarks/apfel_test_0.mp4_landmarks.csv"
I0000 00:00:1749821393.258619 55382079 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1749821393.264307 55647107 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback te

Processed kiwi_test_1.mp4 frame 6
Processed kiwi_test_1.mp4 frame 7
Processed kiwi_test_1.mp4 frame 8
Processed kiwi_test_1.mp4 frame 9
Processed kiwi_test_1.mp4 frame 10
Processed kiwi_test_1.mp4 frame 11
Processed kiwi_test_1.mp4 frame 12
Processed kiwi_test_1.mp4 frame 13
Processed kiwi_test_1.mp4 frame 14
Processed kiwi_test_1.mp4 frame 15
Processed kiwi_test_1.mp4 frame 16
Processed kiwi_test_1.mp4 frame 17
Processed kiwi_test_1.mp4 frame 18
Processed kiwi_test_1.mp4 frame 19
Processed kiwi_test_1.mp4 frame 20
Processed kiwi_test_1.mp4 frame 21
Processed kiwi_test_1.mp4 frame 22
Processed kiwi_test_1.mp4 frame 23
Processed kiwi_test_1.mp4 frame 24
Processed kiwi_test_1.mp4 frame 25
Processed kiwi_test_1.mp4 frame 26
Processed kiwi_test_1.mp4 frame 27
Processed kiwi_test_1.mp4 frame 28
Processed kiwi_test_1.mp4 frame 29
Processed kiwi_test_1.mp4 frame 30
Processed kiwi_test_1.mp4 frame 31
Processed kiwi_test_1.mp4 frame 32
Processed kiwi_test_1.mp4 frame 33
Processed kiwi_test_1.mp

I0000 00:00:1749821394.341393 55382079 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1749821394.346321 55647189 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749821394.350374 55647194 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processed kiwi_test_0.mp4 frame 3
Processed kiwi_test_0.mp4 frame 4
Processed kiwi_test_0.mp4 frame 5
Processed kiwi_test_0.mp4 frame 6
Processed kiwi_test_0.mp4 frame 7
Processed kiwi_test_0.mp4 frame 8
Processed kiwi_test_0.mp4 frame 9
Processed kiwi_test_0.mp4 frame 10
Processed kiwi_test_0.mp4 frame 11
Processed kiwi_test_0.mp4 frame 12
Processed kiwi_test_0.mp4 frame 13
Processed kiwi_test_0.mp4 frame 14
Processed kiwi_test_0.mp4 frame 15
Processed kiwi_test_0.mp4 frame 16
Processed kiwi_test_0.mp4 frame 17
Processed kiwi_test_0.mp4 frame 18
Processed kiwi_test_0.mp4 frame 19
Processed kiwi_test_0.mp4 frame 20
Processed kiwi_test_0.mp4 frame 21
Processed kiwi_test_0.mp4 frame 22
Processed kiwi_test_0.mp4 frame 23
Processed kiwi_test_0.mp4 frame 24
Processed kiwi_test_0.mp4 frame 25
Processed kiwi_test_0.mp4 frame 26
Processed kiwi_test_0.mp4 frame 27
Processed kiwi_test_0.mp4 frame 28
Processed kiwi_test_0.mp4 frame 29
Processed kiwi_test_0.mp4 frame 30
Processed kiwi_test_0.mp4 f

I0000 00:00:1749821394.868818 55382079 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1749821394.874102 55647229 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749821394.881285 55647239 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processed banane_test_1.mp4 frame 10
Processed banane_test_1.mp4 frame 11
Processed banane_test_1.mp4 frame 12
Processed banane_test_1.mp4 frame 13
Processed banane_test_1.mp4 frame 14
Processed banane_test_1.mp4 frame 15
Processed banane_test_1.mp4 frame 16
Processed banane_test_1.mp4 frame 17
Processed banane_test_1.mp4 frame 18
Processed banane_test_1.mp4 frame 19
Processed banane_test_1.mp4 frame 20
Processed banane_test_1.mp4 frame 21
Processed banane_test_1.mp4 frame 22
